In [1]:
import numpy as np
import pandas as pd


In [2]:
import pandas as pd

data = {
    "Brand": ["Baskin-Robbins", "Krispy Kreme"],
    
    # Menu Focus
    "Primary_Products": [
        "Ice cream, sundaes, milkshakes, ice cream cakes",
        "Donuts, coffee, limited-time specialty donuts"
    ],
    
    "Menu_Variety_Score": [9, 6],  # subjective scale 1–10
    
    # Pricing Logic
    "Avg_Item_Price_USD": [5.50, 3.25],
    "Premium_Upcharge_Model": [
        "Seasonal flavors, custom cakes",
        "Limited-time donuts, themed collections"
    ],
    
    # Customer Demographics
    "Primary_Age_Range": [
        "Families, teens, young adults (6–35)",
        "Young adults, commuters, office workers (18–45)"
    ],
    
    "Customer_Type": [
        "Family-oriented, celebration-driven",
        "Impulse buyers, routine snack consumers"
    ],
    
    # Business Logic
    "Peak_Traffic_Times": [
        "Evenings, weekends, summer months",
        "Morning commute, promotions, holidays"
    ],
    
    "Impulse_Purchase_Level": [6, 9],  # 1–10 scale
    
    # Revenue Estimates (Per Store)
    "Estimated_Weekly_Revenue_USD": [18000, 25000],
    "Estimated_Yearly_Revenue_USD": [936000, 1300000],
    
    # Profit Margins
    "Estimated_Profit_Margin": [0.18, 0.22],
}

df = pd.DataFrame(data)
df


,Brand,Primary_Products,Menu_Variety_Score,Avg_Item_Price_USD,Premium_Upcharge_Model,Primary_Age_Range,Customer_Type,Peak_Traffic_Times,Impulse_Purchase_Level,Estimated_Weekly_Revenue_USD,Estimated_Yearly_Revenue_USD,Estimated_Profit_Margin
0,Baskin-Robbins,"Ice cream, sundaes, milkshakes, ice cream cakes",9,5.50,"Seasonal flavors, custom cakes","Families, teens, young adults (6–35)","Family-oriented, celebration-driven","Evenings, weekends, summer months",6,18000,936000,0.18
1,Krispy Kreme,"Donuts, coffee, limited-time specialty donuts",6,3.25,"Limited-time donuts, themed collections","Young adults, commuters, office workers (18–45)","Impulse buyers, routine snack consumers","Morning commute, promotions, holidays",9,25000,1300000,0.22


In [4]:
import pandas as pd

def build_weekly_store_econ(
    items_df,
    regions_df,
    base_weekly_tx,
    mix_weights,
    summer_factor=1.0
):
    rows = []

    for _, r in regions_df.iterrows():
        for brand in items_df["brand"].unique():

            # Guard: skip brands not in base tx assumptions
            if brand not in base_weekly_tx:
                continue

            brand_items = items_df[items_df["brand"] == brand]
            base_tx = base_weekly_tx[brand] * r["demand_mult"]

            for _, it in brand_items.iterrows():
                weight = mix_weights.get(brand, {}).get(it["item_name"], 0.0)
                if weight == 0:
                    continue

                item_tx = base_tx * weight

                # Seasonality
                seasonal_mult = 1 + (summer_factor - 1) * it["seasonality_summer_pull"]

                price = it["avg_price_usd"] * r["price_mult"]
                revenue = item_tx * price * seasonal_mult

                # Margin adjustment for opex pressure
                base_margin = it["gross_margin_pct_est"]
                opex_penalty = 0.03 * max(r["opex_mult"] - 1, 0)
                adj_margin = max(base_margin - opex_penalty, 0.05)

                profit = revenue * adj_margin * r["tax_mult"]

                rows.append({
                    "region": r["region"],
                    "brand": brand,
                    "item_name": it["item_name"],
                    "category": it["category"],
                    "weekly_transactions_est": item_tx * seasonal_mult,
                    "avg_price_local_indexed": price,
                    "weekly_revenue_usd_est": revenue,
                    "weekly_profit_usd_est": profit,
                    "margin_pct_est": adj_margin
                })

    return pd.DataFrame(rows)


In [7]:
import pandas as pd

# --------------------------------------------------
# STEP 1: Create the base weekly brand-region dataset
# --------------------------------------------------
weekly_brand_region = pd.DataFrame({
    "region": ["NA", "NA", "EU", "EU"],
    "brand": ["BrandA", "BrandB", "BrandA", "BrandB"],
    "weekly_revenue_usd_est": [1_000_000, 800_000, 700_000, 600_000],
    "weekly_profit_usd_est": [220_000, 160_000, 140_000, 120_000]
})

# --------------------------------------------------
# STEP 2: Projection function
# --------------------------------------------------
def multi_year_projection(
    base_brand_region_df,
    years=5,
    start_year=2026,
    demand_growth=0.02,
    price_inflation=0.03,
    margin_drift=-0.001
):
    """
    Expects columns:
      - region
      - brand
      - weekly_revenue_usd_est
      - weekly_profit_usd_est
    """

    out = []

    for _, row in base_brand_region_df.iterrows():
        region = row["region"]
        brand = row["brand"]

        base_week_rev = row["weekly_revenue_usd_est"]
        base_week_profit = row["weekly_profit_usd_est"]

        # Safe base margin
        base_margin = base_week_profit / max(base_week_rev, 1e-9)

        for i in range(years):
            year = start_year + i

            demand_mult = (1 + demand_growth) ** i
            price_mult = (1 + price_inflation) ** i

            # Margin erosion with floor
            margin = max(base_margin + (margin_drift * i), 0.05)

            week_rev = base_week_rev * demand_mult * price_mult
            week_profit = week_rev * margin

            out.append({
                "year": year,
                "region": region,
                "brand": brand,
                "proj_weekly_revenue_usd": week_rev,
                "proj_weekly_profit_usd": week_profit,
                "proj_yearly_revenue_usd": week_rev * 52,
                "proj_yearly_profit_usd": week_profit * 52,
                "proj_margin_pct": margin
            })

    return pd.DataFrame(out)

# --------------------------------------------------
# STEP 3: Run projection
# --------------------------------------------------
projection_df = multi_year_projection(
    base_brand_region_df=weekly_brand_region,
    years=7,
    start_year=2026,
    demand_growth=0.025,
    price_inflation=0.03,
    margin_drift=-0.0005
)

# --------------------------------------------------
# STEP 4: Inspect results
# --------------------------------------------------
print(projection_df.head())


   year region   brand  proj_weekly_revenue_usd  proj_weekly_profit_usd  \
0  2026     NA  BrandA             1.000000e+06           220000.000000   
1  2027     NA  BrandA             1.055750e+06           231737.125000   
2  2028     NA  BrandA             1.114608e+06           244099.165688   
3  2029     NA  BrandA             1.176747e+06           257119.320444   
4  2030     NA  BrandA             1.242351e+06           270832.546992   

   proj_yearly_revenue_usd  proj_yearly_profit_usd  proj_margin_pct  
0             5.200000e+07            1.144000e+07           0.2200  
1             5.489900e+07            1.205033e+07           0.2195  
2             5.795962e+07            1.269316e+07           0.2190  
3             6.119087e+07            1.337020e+07           0.2185  
4             6.460226e+07            1.408329e+07           0.2180  


In [10]:
import pandas as pd

# -----------------------------
# Create base data (example)
# -----------------------------
weekly_item_econ = pd.DataFrame({
    "item_name": ["Original Glazed", "Chocolate Cake"],
    "brand": ["Krispy Kreme", "Krispy Kreme"],
    "category": ["Beverage", "Cake"],
    "region": ["US-East", "US-West"],
    "weekly_revenue_usd_est": [1200, 900],
    "weekly_profit_usd_est": [350, 280]
})

regions_df = pd.DataFrame({
    "region": ["US-East", "US-West"],
    "price_mult": [1.0, 1.1],
    "demand_mult": [1.05, 0.95],
    "opex_mult": [1.0, 1.15]
})

# -----------------------------
# Feature engineering
# -----------------------------
ml_df = weekly_item_econ.copy()

ml_df["is_premium_item"] = ml_df["category"].isin(
    ["Cake", "Seasonal", "Box"]
).astype(int)

ml_df["is_morning_biased"] = (
    (ml_df["category"] == "Beverage") |
    (ml_df["brand"] == "Krispy Kreme")
).astype(int)

# Merge region multipliers
ml_df = ml_df.merge(
    regions_df,
    on="region",
    how="left"
)

# One-hot encoding
ml_ready = pd.get_dummies(
    ml_df,
    columns=["brand", "category", "region"],
    drop_first=False
)

# -----------------------------
# ML matrices
# -----------------------------
target = "weekly_profit_usd_est"

drop_cols = [
    "item_name",
    "weekly_revenue_usd_est",
    "weekly_profit_usd_est"
]

feature_cols = [c for c in ml_ready.columns if c not in drop_cols]

X = ml_ready[feature_cols]
y = ml_ready[target]

# -----------------------------
# Preview
# -----------------------------
print(X.head())
print(y.head())



   is_premium_item  is_morning_biased  price_mult  demand_mult  opex_mult  \
0                0                  1         1.0         1.05       1.00   
1                1                  1         1.1         0.95       1.15   

   brand_Krispy Kreme  category_Beverage  category_Cake  region_US-East  \
0                True               True          False            True   
1                True              False           True           False   

   region_US-West  
0           False  
1            True  
0    350
1    280
Name: weekly_profit_usd_est, dtype: int64


In [12]:
import pandas as pd

# --------------------------------
# Define items_df (base items table)
# --------------------------------
items_df = pd.DataFrame({
    "item_name": [
        "Single Scoop",
        "Double Scoop",
        "Classic Sundae",
        "Milkshake",
        "Ice Cream Cake (8in)",
        "Limited Flavor Pint",
        "Original Glazed (Single)",
        "Specialty Donut",
        "Dozen Original Glazed",
        "Assorted Dozen",
        "Brewed Coffee",
        "Limited Collection (4-pack)"
    ]
})

# --------------------------------
# Demographic affinity map
# --------------------------------
demo_map = {
    "Kids": {
        "Single Scoop":0.9, "Double Scoop":0.6, "Classic Sundae":0.7, "Milkshake":0.6,
        "Ice Cream Cake (8in)":0.4, "Limited Flavor Pint":0.2,
        "Original Glazed (Single)":0.4, "Specialty Donut":0.5,
        "Dozen Original Glazed":0.5, "Assorted Dozen":0.4,
        "Brewed Coffee":0.05, "Limited Collection (4-pack)":0.3
    },
    "Teens": {
        "Single Scoop":0.6, "Double Scoop":0.8, "Classic Sundae":0.7, "Milkshake":0.85,
        "Ice Cream Cake (8in)":0.2, "Limited Flavor Pint":0.4,
        "Original Glazed (Single)":0.5, "Specialty Donut":0.7,
        "Dozen Original Glazed":0.4, "Assorted Dozen":0.4,
        "Brewed Coffee":0.2, "Limited Collection (4-pack)":0.6
    },
    "YoungAdults": {
        "Single Scoop":0.4, "Double Scoop":0.7, "Classic Sundae":0.5, "Milkshake":0.7,
        "Ice Cream Cake (8in)":0.25, "Limited Flavor Pint":0.6,
        "Original Glazed (Single)":0.7, "Specialty Donut":0.8,
        "Dozen Original Glazed":0.5, "Assorted Dozen":0.6,
        "Brewed Coffee":0.9, "Limited Collection (4-pack)":0.75
    },
    "Families": {
        "Single Scoop":0.85, "Double Scoop":0.6, "Classic Sundae":0.7, "Milkshake":0.5,
        "Ice Cream Cake (8in)":0.9, "Limited Flavor Pint":0.35,
        "Original Glazed (Single)":0.5, "Specialty Donut":0.5,
        "Dozen Original Glazed":0.8, "Assorted Dozen":0.7,
        "Brewed Coffee":0.3, "Limited Collection (4-pack)":0.5
    },
    "Commuters": {
        "Single Scoop":0.05, "Double Scoop":0.05, "Classic Sundae":0.05, "Milkshake":0.05,
        "Ice Cream Cake (8in)":0.02, "Limited Flavor Pint":0.02,
        "Original Glazed (Single)":0.9, "Specialty Donut":0.7,
        "Dozen Original Glazed":0.4, "Assorted Dozen":0.35,
        "Brewed Coffee":0.95, "Limited Collection (4-pack)":0.2
    }
}

# --------------------------------
# Apply affinity features
# --------------------------------
for seg in demo_map:
    items_df[f"{seg}_affinity"] = items_df["item_name"].map(
        lambda x: demo_map[seg].get(x, 0.1)
    )

# Preview result
items_df



,item_name,Kids_affinity,Teens_affinity,YoungAdults_affinity,Families_affinity,Commuters_affinity
0,Single Scoop,0.90,0.60,0.40,0.85,0.05
1,Double Scoop,0.60,0.80,0.70,0.60,0.05
2,Classic Sundae,0.70,0.70,0.50,0.70,0.05
3,Milkshake,0.60,0.85,0.70,0.50,0.05
4,Ice Cream Cake (8in),0.40,0.20,0.25,0.90,0.02
5,Limited Flavor Pint,0.20,0.40,0.60,0.35,0.02
6,Original Glazed (Single),0.40,0.50,0.70,0.50,0.90
7,Specialty Donut,0.50,0.70,0.80,0.50,0.70
8,Dozen Original Glazed,0.50,0.40,0.50,0.80,0.40
9,Assorted Dozen,0.40,0.40,0.60,0.70,0.35
